# Best Model

In [44]:
# Import necessary libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
import warnings
warnings.filterwarnings("ignore")

In [45]:
# Directory paths
train_dir = 'dataset/train'
validation_dir = 'dataset/validation'
test_dir = 'dataset/test'

In [46]:
# Parameters
img_height, img_width = 150, 150
batch_size = 32

# Data augmentation for training set with enhanced parameters
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,           # Increased rotation range for more variety
    width_shift_range=0.3,       # Increased width shift
    height_shift_range=0.3,      # Increased height shift
    shear_range=0.3,             # Increased shear range
    zoom_range=0.3,              # Increased zoom range
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data normalization for validation and test sets
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Train generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Validation generator
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 23972 images belonging to 30 classes.
Found 3030 images belonging to 30 classes.
Found 2998 images belonging to 30 classes.


In [47]:
# Use Transfer Learning with VGG16 (pre-trained on ImageNet)
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the base model layers (can unfreeze some for fine-tuning later)
for layer in vgg_base.layers[:15]:  # Freeze first 15 layers
    layer.trainable = False

# Build the new model on top of VGG16
model = Sequential([
    vgg_base,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # Dropout for regularization
    Dense(256, activation='relu'),
    Dropout(0.5),  # Additional dropout
    BatchNormalization(),  # Batch Normalization for stability
    Dense(train_generator.num_classes, activation='softmax')  # Output layer
])

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001),  # Lower learning rate for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 4, 4, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 512)                 │       4,194,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 30)                  │           7,710 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,049,566 (72.67 MB)

 Trainable params: 11,413,790 (43.54 MB)

 Non-trainable params: 7,635,776 (29.13 MB)

In [48]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
# Model checkpoint to save the best model
checkpoint = ModelCheckpoint('model2.keras', monitor='val_loss', save_best_only=True)
# Learning rate reduction on plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

In [49]:
# Train the model with validation data
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=30,  # Increased number of epochs
    callbacks=[early_stopping, checkpoint, reduce_lr]  # Added reduce_lr
)

Epoch 1/30
749/749 ━━━━━━━━━━━━━━━━━━━━ 2364s 3s/step - accuracy: 0.0438 - loss: 4.0445 - val_accuracy: 0.2380 - val_loss: 2.8078 - learning_rate: 1.0000e-05
Epoch 2/30
749/749 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.0312 - loss: 3.8671 - val_accuracy: 0.2273 - val_loss: 2.7739 - learning_rate: 1.0000e-05
Epoch 3/30
749/749 ━━━━━━━━━━━━━━━━━━━━ 2307s 3s/step - accuracy: 0.1002 - loss: 3.5017 - val_accuracy: 0.3833 - val_loss: 2.2927 - learning_rate: 1.0000e-05
Epoch 4/30
749/749 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.0938 - loss: 3.1971 - val_accuracy: 0.3182 - val_loss: 2.6187 - learning_rate: 1.0000e-05
Epoch 5/30
749/749 ━━━━━━━━━━━━━━━━━━━━ 2319s 3s/step - accuracy: 0.1641 - loss: 3.0506 - val_accuracy: 0.4641 - val_loss: 1.9756 - learning_rate: 1.0000e-05
Epoch 6/30
749/749 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.3125 - loss: 2.5436 - val_accuracy: 0.4545 - val_loss: 2.0845 - learning_rate: 1.0000e-05
Epoch 7/30
749/749 ━━━━━━━━━━━━━━━━━━━━ 2305s 3s/step - ac

In [50]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}')

94/94 ━━━━━━━━━━━━━━━━━━━━ 207s 2s/step - accuracy: 0.7396 - loss: 0.9748
Test Accuracy: 0.7415, Test Loss: 0.9500


# Previous Model

In [31]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")

In [23]:
# Directory paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Parameters
img_height, img_width = 150, 150
batch_size = 32

# Data augmentation for training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data normalization for test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 23972 images belonging to 30 classes.
Found 2998 images belonging to 30 classes.


In [3]:
# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # 30 classes
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │          15,390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,049,054 (72.67 MB)

 Trainable params: 19,049,054 (72.67 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,  # Set the number of epochs based on your dataset size
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=test_generator.samples // batch_size
)

Epoch 1/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 1129s 1s/step - accuracy: 0.1524 - loss: 2.8881 - val_accuracy: 0.3589 - val_loss: 2.0210
Epoch 2/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 1s 343us/step - accuracy: 0.4062 - loss: 1.8914 - val_accuracy: 0.4091 - val_loss: 2.1891
Epoch 3/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 748s 993ms/step - accuracy: 0.3440 - loss: 2.1036 - val_accuracy: 0.4402 - val_loss: 1.7842
Epoch 4/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4062 - loss: 1.8129 - val_accuracy: 0.5000 - val_loss: 1.7633
Epoch 5/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 783s 1s/step - accuracy: 0.4377 - loss: 1.7962 - val_accuracy: 0.5591 - val_loss: 1.4375
Epoch 6/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 1s 230us/step - accuracy: 0.4688 - loss: 1.3397 - val_accuracy: 0.5455 - val_loss: 1.4420
Epoch 7/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 802s 1s/step - accuracy: 0.5029 - loss: 1.5885 - val_accuracy: 0.5444 - val_loss: 1.4029
Epoch 8/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 1s 146us/step - accuracy: 0.4688 - loss: 1.5441 - va

In [14]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test accuracy: {test_acc}")

93/93 ━━━━━━━━━━━━━━━━━━━━ 17s 182ms/step - accuracy: 0.6065 - loss: 1.2764
Test accuracy: 0.6149193644523621


In [15]:
# Predict on test set
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

94/94 ━━━━━━━━━━━━━━━━━━━━ 35s 371ms/step


In [17]:
print('Classification Report')
target_names = list(train_generator.class_indices.keys())
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Classification Report
                precision    recall  f1-score   support

      aloevera       0.39      0.66      0.49       100
        banana       0.85      0.56      0.67       100
       bilimbi       0.48      0.43      0.46       100
    cantaloupe       0.50      0.54      0.52       100
       cassava       0.60      0.68      0.64       100
       coconut       0.40      0.66      0.50       100
          corn       0.68      0.55      0.61       100
      cucumber       0.66      0.55      0.60        99
       curcuma       0.73      0.70      0.71       100
      eggplant       0.74      0.93      0.83       100
      galangal       0.46      0.65      0.54       100
        ginger       0.63      0.33      0.43       100
         guava       0.88      0.65      0.75       100
          kale       0.38      0.90      0.53       100
     longbeans       0.72      0.66      0.69       100
         mango       0.54      0.42      0.47       100
         melon       0.55

In [6]:
# Save the model
model.save('plant_classifier_model.h5')